## Crawl Tweets

In [ ]:
!pip install tweepy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import json
from tweepy import OAuthHandler, API, Cursor

In [ ]:
f = open('twitter_token.json','r')
token = json.load(f)
f.close()

In [ ]:
access_token = token['Access Token']
access_secret = token['Access Token Secret']
consumer_key = token['Consumer Key']
consumer_secret = token['Consumer Key Secret']

auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = API(auth, wait_on_rate_limit = True, timeout = 15*60)

In [ ]:
## Jika terdapat lebbih dari 100 tweet dan kita ingin dapatkan semua tweet tersebut, bisa pakai fungsi/method Cursor dari tweepy

tweets_all = []

## keyword yg akan kita cari dari sebuah tweet
query = '"resesi 2023"'

count = 1
for tweet_list in Cursor(api.search, query, count=100, tweet_mode='extended',lang='id').pages():
    tweets_all.extend([i._json for i in tweet_list])
    print('request ke:',count)
    count+=1

request ke: 1
request ke: 2
request ke: 3
request ke: 4
request ke: 5
request ke: 6
request ke: 7
request ke: 8
request ke: 9
request ke: 10
request ke: 11
request ke: 12
request ke: 13
request ke: 14
request ke: 15
request ke: 16
request ke: 17
request ke: 18
request ke: 19
request ke: 20
request ke: 21
request ke: 22
request ke: 23
request ke: 24
request ke: 25
request ke: 26
request ke: 27
request ke: 28
request ke: 29
request ke: 30
request ke: 31
request ke: 32
request ke: 33
request ke: 34
request ke: 35
request ke: 36
request ke: 37
request ke: 38
request ke: 39
request ke: 40
request ke: 41
request ke: 42
request ke: 43
request ke: 44
request ke: 45
request ke: 46
request ke: 47
request ke: 48
request ke: 49
request ke: 50
request ke: 51
request ke: 52
request ke: 53
request ke: 54
request ke: 55
request ke: 56
request ke: 57
request ke: 58
request ke: 59
request ke: 60
request ke: 61
request ke: 62
request ke: 63
request ke: 64
request ke: 65
request ke: 66
request ke: 67
requ

In [ ]:
tweets_all[0]

{'created_at': 'Wed Oct 19 13:03:44 +0000 2022',
 'id': 1582719144012972032,
 'id_str': '1582719144012972032',
 'full_text': '@TXTdariHunter Siap menghadapi resesi 2023',
 'truncated': False,
 'display_text_range': [15, 42],
 'entities': {'hashtags': [],
  'symbols': [],
  'user_mentions': [{'screen_name': 'TXTdariHunter',
    'name': '𝕋𝕠𝕟𝕘𝕜𝕣𝕠𝕟𝕘𝕒𝕟 ℍ𝕦𝕟𝕥𝕖𝕣 ☕',
    'id': 1376684644943327236,
    'id_str': '1376684644943327236',
    'indices': [0, 14]}],
  'urls': []},
 'metadata': {'iso_language_code': 'in', 'result_type': 'recent'},
 'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>',
 'in_reply_to_status_id': 1582718217486684160,
 'in_reply_to_status_id_str': '1582718217486684160',
 'in_reply_to_user_id': 1376684644943327236,
 'in_reply_to_user_id_str': '1376684644943327236',
 'in_reply_to_screen_name': 'TXTdariHunter',
 'user': {'id': 1250812517141475328,
  'id_str': '1250812517141475328',
  'name': 'InekeXa ☀️☀️ ⚡️Freetag⚡️☀️☀️',
  'screen

In [ ]:
with open('tweets_resesi_2023.json','w') as f:
    json.dump(tweets_all,f)

### Kalau belum ada twitter API nya, bisa langsung ke generate data SNA nya saja

## Generate data SNA

In [5]:
import pandas as pd
from datetime import datetime, timedelta
from google.colab import drive
import json

In [3]:
drive.mount('drive')

Mounted at drive


In [8]:
def get_tweet_details(x, tweet_list):
    
    user = x['user']['screen_name']
    user_id = x['user']['id_str']
    _id = x['id_str']
    date = x['created_at']
    text = x['full_text']
    post_url = 'https://twitter.com/' + user + '/status/' + _id
    
            
    dct = {'source':user, 'user_id':user_id, 'tweet_id':_id, 'created_at':date, 'text':text, 'post_url':post_url, 
           'target':'', 'target_user_id':'', 'target_tweet':''}
    
    if 'retweeted_status' in x.keys():
        target_user = x['retweeted_status']['user']['screen_name']
        target_tweet = x['retweeted_status']['id_str']
        dct['target'] = target_user
        dct['target_user_id'] = x['retweeted_status']['user']['id_str']
        dct['target_tweet'] = 'https://twitter.com/{}/status/{}'.format(target_user, str(target_tweet))
        dct['tweet_type'] = 'Retweet'
        tweet_list.append(dct)
    
    elif 'quoted_status' in x.keys():
        target_user = x['quoted_status']['user']['screen_name']
        target_tweet = x['quoted_status']['id_str']
        dct['target'] = x['quoted_status']['user']['screen_name']
        dct['target_user_id'] = x['quoted_status']['user']['id_str']
        dct['target_tweet'] = 'https://twitter.com/{}/status/{}'.format(target_user, str(target_tweet))
        dct['tweet_type'] = 'Quote'
        tweet_list.append(dct)
        
    if x['in_reply_to_screen_name'] != None:
        target_user = x['in_reply_to_screen_name']
        target_tweet = x['in_reply_to_status_id_str']
        dct['target'] = target_user
        dct['target_user_id'] = x['in_reply_to_user_id_str']
        dct['target_tweet'] = 'https://twitter.com/{}/status/{}'.format(target_user, str(target_tweet))
        dct['tweet_type'] = 'Reply'
        tweet_list.append(dct)
    
    if all(k not in x.keys() for k in ['retweeted_status','quoted_status']) and x['in_reply_to_screen_name'] == None:
        dct['tweet_type'] = 'Tweet'
        tweet_list.append(dct)

In [9]:
data = []

with open('/content/drive/MyDrive/Intro to SNA Pacmann/tweets_resesi_2023.json','r') as f:
    data = json.load(f)

In [10]:
len(data)

12980

In [11]:
tweets = []

for elem in data:
    get_tweet_details(elem,tweets)

In [12]:
tweets[0]

{'source': 'Bles2edByLord',
 'user_id': '1250812517141475328',
 'tweet_id': '1582719144012972032',
 'created_at': 'Wed Oct 19 13:03:44 +0000 2022',
 'text': '@TXTdariHunter Siap menghadapi resesi 2023',
 'post_url': 'https://twitter.com/Bles2edByLord/status/1582719144012972032',
 'target': 'TXTdariHunter',
 'target_user_id': '1376684644943327236',
 'target_tweet': 'https://twitter.com/TXTdariHunter/status/1582718217486684160',
 'tweet_type': 'Reply'}

In [13]:
tweets_df = pd.json_normalize(tweets)
tweets_df.head()

,source,user_id,tweet_id,created_at,text,post_url,target,target_user_id,target_tweet,tweet_type
0,Bles2edByLord,1250812517141475328,1582719144012972032,Wed Oct 19 13:03:44 +0000 2022,@TXTdariHunter Siap menghadapi resesi 2023,https://twitter.com/Bles2edByLord/status/15827...,TXTdariHunter,1376684644943327236,https://twitter.com/TXTdariHunter/status/15827...,Reply
1,BrunellaBiebie,751434590,1582718411557474307,Wed Oct 19 13:00:49 +0000 2022,"RT @ainunrozi: jangankan ancaman resesi 2023, ...",https://twitter.com/BrunellaBiebie/status/1582...,ainunrozi,183585238,https://twitter.com/ainunrozi/status/158265189...,Retweet
2,vfsfebry,1092221130,1582717918466670592,Wed Oct 19 12:58:51 +0000 2022,"RT @ainunrozi: jangankan ancaman resesi 2023, ...",https://twitter.com/vfsfebry/status/1582717918...,ainunrozi,183585238,https://twitter.com/ainunrozi/status/158265189...,Retweet
3,shanindya001,1163423607214575618,1582716965147189249,Wed Oct 19 12:55:04 +0000 2022,"RT @ainunrozi: jangankan ancaman resesi 2023, ...",https://twitter.com/shanindya001/status/158271...,ainunrozi,183585238,https://twitter.com/ainunrozi/status/158265189...,Retweet
4,anginmei,969553520,1582716923485491200,Wed Oct 19 12:54:54 +0000 2022,"RT @ainunrozi: jangankan ancaman resesi 2023, ...",https://twitter.com/anginmei/status/1582716923...,ainunrozi,183585238,https://twitter.com/ainunrozi/status/158265189...,Retweet


In [14]:
month_dct = {'Jan':'01','Feb':'02','Mar':'03','Apr':'04',"May":'05','Jun':'06','Jul':'07','Aug':'08','Sep':'09','Oct':'10',
            'Nov':'11','Dec':'12'}

def get_created_at(x):
    
    _ls = x.split()
    date = _ls[2]
    month = _ls[1]
    month = month_dct.get(month)
    year = _ls[-1]
        
    new_date = ('-').join([year,month,date])
    new_date = new_date + ' ' + _ls[3][:8]
    new_date = datetime.strptime(new_date, '%Y-%m-%d %H:%M:%S')+timedelta(hours=7)
    
    return new_date

In [15]:
tweets_df['created_at'] = tweets_df['created_at'].apply(lambda x: get_created_at(x))
tweets_df.head()

,source,user_id,tweet_id,created_at,text,post_url,target,target_user_id,target_tweet,tweet_type
0,Bles2edByLord,1250812517141475328,1582719144012972032,2022-10-19 20:03:44,@TXTdariHunter Siap menghadapi resesi 2023,https://twitter.com/Bles2edByLord/status/15827...,TXTdariHunter,1376684644943327236,https://twitter.com/TXTdariHunter/status/15827...,Reply
1,BrunellaBiebie,751434590,1582718411557474307,2022-10-19 20:00:49,"RT @ainunrozi: jangankan ancaman resesi 2023, ...",https://twitter.com/BrunellaBiebie/status/1582...,ainunrozi,183585238,https://twitter.com/ainunrozi/status/158265189...,Retweet
2,vfsfebry,1092221130,1582717918466670592,2022-10-19 19:58:51,"RT @ainunrozi: jangankan ancaman resesi 2023, ...",https://twitter.com/vfsfebry/status/1582717918...,ainunrozi,183585238,https://twitter.com/ainunrozi/status/158265189...,Retweet
3,shanindya001,1163423607214575618,1582716965147189249,2022-10-19 19:55:04,"RT @ainunrozi: jangankan ancaman resesi 2023, ...",https://twitter.com/shanindya001/status/158271...,ainunrozi,183585238,https://twitter.com/ainunrozi/status/158265189...,Retweet
4,anginmei,969553520,1582716923485491200,2022-10-19 19:54:54,"RT @ainunrozi: jangankan ancaman resesi 2023, ...",https://twitter.com/anginmei/status/1582716923...,ainunrozi,183585238,https://twitter.com/ainunrozi/status/158265189...,Retweet


In [23]:
edge = tweets_df.copy()
edge.columns = ['Source Label', 'Source', 'Tweet Id','Created at','Text','Post Url','Target Label','Target','Target Tweet','Tweet Type']
edge = edge.sort_values('Created at').reset_index(drop=True)
edge = edge[(edge['Created at']>=datetime.strptime("2022-10-17",'%Y-%m-%d')) & (edge['Created at']<datetime.strptime('2022-10-19','%Y-%m-%d'))].reset_index(drop=True)
edge

,Source Label,Source,Tweet Id,Created at,Text,Post Url,Target Label,Target,Target Tweet,Tweet Type
0,Mizzumono,1149875459502403584,1581691649943797761,2022-10-17 00:00:50,RT @tanganbelang_tb: Untuk mempersiapkan reses...,https://twitter.com/Mizzumono/status/158169164...,tanganbelang_tb,3004551331,https://twitter.com/tanganbelang_tb/status/158...,Retweet
1,hangrydsaster,546056887,1581691863685533697,2022-10-17 00:01:41,RT @tanganbelang_tb: Untuk mempersiapkan reses...,https://twitter.com/hangrydsaster/status/15816...,tanganbelang_tb,3004551331,https://twitter.com/tanganbelang_tb/status/158...,Retweet
2,dhiraap,124693421,1581691882534363136,2022-10-17 00:01:45,RT @tanganbelang_tb: Untuk mempersiapkan reses...,https://twitter.com/dhiraap/status/15816918825...,tanganbelang_tb,3004551331,https://twitter.com/tanganbelang_tb/status/158...,Retweet
3,sielynnee,1563588514159005698,1581691974482268160,2022-10-17 00:02:07,RT @tanganbelang_tb: Untuk mempersiapkan reses...,https://twitter.com/sielynnee/status/158169197...,tanganbelang_tb,3004551331,https://twitter.com/tanganbelang_tb/status/158...,Retweet
4,miftahfaizal,894837626,1581691985580429314,2022-10-17 00:02:10,RT @tanganbelang_tb: Untuk mempersiapkan reses...,https://twitter.com/miftahfaizal/status/158169...,tanganbelang_tb,3004551331,https://twitter.com/tanganbelang_tb/status/158...,Retweet
...,...,...,...,...,...,...,...,...,...,...
1755,pndrbx23,1301486057301528577,1582413851408355328,2022-10-18 23:50:36,@M4NV514_131454 @carfiqu @lndomieee @herbertus...,https://twitter.com/pndrbx23/status/1582413851...,M4NV514_131454,635870003,https://twitter.com/M4NV514_131454/status/1582...,Reply
1756,zaidsyafire,2250211291,1582413911114276864,2022-10-18 23:50:50,"2023 bakal gelap, 2023 bakal resesi, 2023 baka...",https://twitter.com/zaidsyafire/status/1582413...,,,,Tweet
1757,uciyynrl,1465196591631908865,1582414729561051136,2022-10-18 23:54:06,RT @tanganbelang_tb: Untuk mempersiapkan reses...,https://twitter.com/uciyynrl/status/1582414729...,tanganbelang_tb,3004551331,https://twitter.com/tanganbelang_tb/status/158...,Retweet
1758,Gardapengawalp,1574419211501203456,1582414933928513541,2022-10-18 23:54:54,@AzwarEko @Tan_Mar3M @msaid_didu @aniesbasweda...,https://twitter.com/Gardapengawalp/status/1582...,Gardapengawalp,1574419211501203456,https://twitter.com/Gardapengawalp/status/1582...,Reply


In [24]:
node_ls_id = edge['Source'].tolist() + edge['Target'].tolist()
node_ls_label = edge['Source Label'].tolist() + edge['Target Label'].tolist()

node = pd.DataFrame(list(zip(node_ls_id,node_ls_label)),columns=['Id','Label'])
node

,Id,Label
0,1149875459502403584,Mizzumono
1,546056887,hangrydsaster
2,124693421,dhiraap
3,1563588514159005698,sielynnee
4,894837626,miftahfaizal
...,...,...
3515,635870003,M4NV514_131454
3516,,
3517,3004551331,tanganbelang_tb
3518,1574419211501203456,Gardapengawalp


In [25]:
edge.to_csv('/content/drive/MyDrive/Intro to SNA Pacmann/SNA_Twitter_resesi_2023_edge.csv',index=False)
node.to_csv('/content/drive/MyDrive/Intro to SNA Pacmann/SNA_Twitter_resesi_2023_node.csv',index=False)